In [1]:
import pandas as pd
import numpy as np
import os
import random

In [2]:
# set locations for working files
# ATTPCROOTv2 directory
attpcroot_dir = '/mnt/analysis/e17023/Adam/ATTPCROOTv2/'

# Automation directory
automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/'

In [3]:
def indicator_file(file_type, indicator_directory=automation_dir):
    df = pd.DataFrame([0])
    df.to_csv(indicator_directory + file_type + '.csv', index=False)
    print(file_type + ' FILE CREATED')

In [4]:
def energy_to_momentum(energy, particle):
    # input energy in KeV, convert to MeV
    energy = energy/1000

    # Mass values from NIST
    if particle == 'a':
        mass = 3727.3794066 # MeV/c^2
    elif particle == 'p':
        mass = 938.27208816 # MeV/c^2
    else:
        indicator_file('STOP')
        raise Exception('Error: particle must be "a" or "p"')
    momentum = np.sqrt(2*mass*energy)/1000 # GeV/c
    return momentum